#### Glossary Terms Bulk Upload
This notebook demonstrates how to bulk upload a set of Glossary Terms from a csv file.  This example includes code to synthetically create a number of Glossary terms.

#### Configuration

In [ ]:
import yaml, requests, json

# Open the config file to read the client details
with open("../config/config.yaml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

client_id = config[0]['client_id']
client_secret = config[0]['client_secret']
tenant_id = config[0]['tenant_id']
purview_account_name = config[0]['purview_account_name']
scope = f"{config[0]['resource']}/.default"
purview_endpoint = f"https://{purview_account_name}.purview.azure.com"
authority = f"https://login.microsoftonline.com/{tenant_id}"

access_token = ""

#### Functions

In [ ]:
# Function to construct header for HTTP requests
# Parameters:  None
# Returns: Dictionary with header information
##
def getHeaders():
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    return headers;

#### OAuth Login
Perform Authentication using the Microsoft Authentication Library and get a bearer token for subsequent API calls

In [ ]:
from msal import ConfidentialClientApplication

app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)

access_token = result['access_token']

#### Initial Setup
Create a Glossary for the Terms we are going to work with

In [ ]:
# Initial setup:  Create the Glossary and get a reference to it's guid
glossary_name = "Example Glossary"

glossary_uri = f"{purview_endpoint}/catalog/api/atlas/v2/glossary"
payload = json.dumps({
    "name": f"{glossary_name}",
    "shortDescription": "Glossary to test API Integration",
    "longDescription": "Glossary to test API Integration",
    "Language": "en",
    "usage": "Used to test API calls"
})
response = json.loads(requests.request("POST", glossary_uri, headers=getHeaders(), data=payload).content)

glossary_guid = response['guid']
print(response)

#### Create the template file and populuate it with some sample data
We'll use the 'System Default' Term Template to create our Glossary Terms.  We can either download the template from the Portal or just manually create it through some basic code.  We can then create some synthetic glossary data to bulk upload.

In [ ]:
import csv
num_rows_to_create = 250

filename = "glossary_template.csv"
file_headers = ["Name", "Nick Name", "Status", "Definition", "Acronymn", "Resources", "Related Terms", "Synonymns", "Stewards", "Experts", "Parent Term Name", "IsDefinitionRichText", "Term Template Names" ]
file_rows = []


# Create a new file and add the header row
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(file_headers)

    for row in range(num_rows_to_create):
        file_rows.append([
            f"Name {row}",
            f"Nick Name {row}",
            "Draft",
            f"Definition {row}",
            f"Acronymn {row}",
            f"Microsoft Purview Project:https://web.purview.azure.com;Azure portal:https://portal.azure.com;",
            "",
            f"Synonymns {row}",
            "",
            "",
            "",
            "false",
            "System default"
        ])
    
    csvwriter.writerows(file_rows)


#### Uploading the Glossary Term csv
Now that we have created the template, we can upload it using the Import Glossary Terms Via CSV API (https://learn.microsoft.com/en-us/rest/api/purview/catalogdataplane/glossary/import-glossary-terms-via-csv-by-glossary-name?tabs=HTTP)

In [ ]:
# Define the URI for the import operation
import_uri = f"{purview_endpoint}/catalog/api/glossary/name/{requests.utils.quote(glossary_name)}/terms/import?includeTermHierarchy=true&api-version=2022-08-01-preview"

payload = {}

# Define a tuple with the file to upload
files = [
    ('file', (filename, open(filename, 'rb'), 'text/csv'))
    ]

# Construct the header but remove the Content-Type
headers = getHeaders()
headers.pop('Content-Type')

response = requests.request("POST", import_uri, headers=headers, data=payload, files=files)

# Get the operation id so we can check the status in the next cell
operation_guid = json.loads(response.content)['id']

print(response.text)

In [ ]:
# Uri to check the status of the import operation
status_uri = f"{purview_endpoint}/catalog/api/glossary/terms/import/{operation_guid}?api-version=2022-08-01-preview"

headers = getHeaders()
response = json.loads(requests.request("GET", status_uri, headers=headers).content)
print(response)

#### Cleanup
Delete the Glossary that was created.  Note, this won't work if you've assigned any of the Terms to entities.

In [ ]:
#Delete the glossary
delete_uri = f"{purview_endpoint}/catalog/api/atlas/v2/glossary/{glossary_guid}"
headers = getHeaders()
requests.request("DELETE", delete_uri, headers=headers)